# Code

In [18]:
from constraint import *
from simpleai.search import CspProblem, backtrack
import pandas as pd
import re

# puzzle = input("Enter an operation of 2 words that should result in another third word\nFor example: ODD+ODD=EVEN")
puzzle = input("Enter an operation of 2 words that should result in another third word\nFor example: ODD+ODD=EVEN")

words_and_operators = re.findall(r'\b\w+\b|[+*/-]', puzzle)     #split words and operators and save in list
words = []                                                      #list for words
operators = []                                                  #list for operators
letters = []                                                    #list for letters
variables = ()                                                  #Tuple for the variables
domains = {}                                                    #Dictionary for the domains

for item in words_and_operators:                                #add words to word list, operators to operator list and distinct letters to letters list
    if item.isalnum():
        words.append(item)
        for letter in item:
            if letter not in letters:
                letters.append(letter)
    else:
        operators.append(item)

starting_letters = set(word[0] for word in words)               #get letters that are at the start of the words
for letter in letters:                                          #arrange domains for all variables
    if letter in starting_letters:
        domains.update({letter: list(range(1, 10))})            #if letter is start of word allow values 1-10
    else:
        domains.update({letter: list(range(0, 10))})            #else allow values 0-10

variables = tuple(letters)                                      #Assign tuple of letters list to variables

def constraint_unique(variables, values):                       #Constraint to see if all variables are unique
    return len(values) == len(set(values))

def constraint_add(variables, values):
    result_str = ''   
    equation = ''                                          #Create empty string for assembling the result into a result
    
    indexes_list = [[values[variables.index(letter)] for letter in word] for word in words] #list with indexes(in variables) for each letter of each word

    for i, word in enumerate(words[:-1]):                   #assemble operation as string e.g 655+655 for ODD+ODD
        equation += ''.join(str(index) for index in indexes_list[i])
        if i < len(words) - 2:
            equation += operators[0]
    result_str = ''.join(str(index) for index in indexes_list[-1]) #Create result string e.g. 1310 for EVEN
    return eval(equation) == int(result_str)

def create_dataframe(dictionary):                           #create dataframe for letters with corresponding digit for result
    keys = list(dictionary.keys())
    values = list(dictionary.values())
    headers = [f"letter {i + 1}" for i in range(len(keys))]  # Assign "letter x" as the header for each column
    df = pd.DataFrame([keys, values], columns=headers)
    return df

constraints = [                                                 #Define constraints
    ((variables), constraint_unique),
    ((variables), constraint_add)
]

problem = CspProblem(variables, domains, constraints)           #Setup problem
print("puzzle:")                                                #print puzzle in nice layout
for i in range(len(words_and_operators) -1):
    print(words_and_operators[i])
print('=')
print(words_and_operators[-1])
output = backtrack(problem)                                     #Backtrack the hell out of it!
if len(output) > 0:                           
    df = create_dataframe(output)                               #Print found solution if there is a solution
    df_string = df.to_string(index=False)
    print("\nSolution:\n",df_string)
else:
    print("No solution found")


puzzle:
ODD
+
ODD
=
EVEN

Solution:
 letter 1 letter 2 letter 3 letter 4 letter 5
       O        D        E        V        N
       6        5        1        3        0


# Report

Setting up the variables and domains were pretty straight forward so I coded them myself. The way off adding the letters to the list and updating them was something I stole from BingAI

Setting up the constraint was more of a challenge since dynamically assembling the string of numbers for each word was not so straight forward. I also got some help from listening to Koen Van Geel when he was explaining how the eval() function can evaluate strings that contain operations.The conversation that led to the result I used is not accessible because I signed up for something with my account to get more chat messages per conversation and now I can't access BingAI for some reason.

But it gave me the following code:
```
def constraint_add(variables, values):
    result_str = ''   
    equation = ''                                          #Create empty string for assembling the result into a result
    
    indexes_list = [[values[variables.index(letter)] for letter in word] for word in words] #list with indexes(in variables) for each letter of each word

    for i, word in enumerate(words[:-1]):                   #assemble operation as string e.g 655+655 for ODD+ODD
        equation += ''.join(str(index) for index in indexes_list[i])
        if i < len(words) - 2:
            equation += operators[0]
    result_str = ''.join(str(index) for index in indexes_list[-1]) #Create result string e.g. 1310 for EVEN
    return eval(equation) == int(result_str)
```
I used BingAI to convert the solution of a puzzle to a DataFrame where the letters are represented with the corresponding values of the solution.
I started with this prompt:
*Can you write me a function that generates a pandas dataframe that has 2 rows, one containing the keys of a dictionary and the other row that contains the values behind the keys*
Which resulted in this code after some more fine tuning questions:
```
def create_dataframe(dictionary):                           #create dataframe for letters with corresponding digit for result
    keys = list(dictionary.keys())
    values = list(dictionary.values())
    headers = [f"letter {i + 1}" for i in range(len(keys))]  # Assign "letter x" as the header for each column
    df = pd.DataFrame([keys, values], columns=headers)
    return df
```

I also used BingAI for some streamlit related problems. Or to find out if certain things are possible within streamlit.